<a href="https://colab.research.google.com/github/jia6776/GoogleMLB-Gemma-Sprint/blob/main/src/data/train/gemma_2b_it_qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 개발 환경 설정

### 1.1 필수 라이브러리 설치하기

In [2]:
!pip install -q -U transformers datasets bitsandbytes peft trl accelerate wandb

### 1.2 Import modules

In [3]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
WANDB_API_KEY = 


### 1.3 Huggingface & WanDB 로그인

In [5]:
import huggingface_hub
import wandb


huggingface_hub.login(token=HF_TOKEN)
wandb.login(key=WANDB_API_KEY)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# 2. Dataset 생성 및 준비

### 2.1 데이터셋 로드

In [6]:
from datasets import load_dataset
dataset = load_dataset("BLACKBUN/old_korean_newspaper_1897_1910_economy_politic_qa")

README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8946 [00:00<?, ? examples/s]

### 2.2 데이터셋 탐색

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 8946
    })
})

### 2.3 데이터셋 예시

In [8]:
dataset['train'][0]

{'Question': '대한국산림협회가 설립된 배경과 그 중요성은 무엇인가요?',
 'Answer': '대한국산림협회는 산림의 중요성을 인식하고,植林(식림)의 필요성을 강조하기 위해 설립되었습니다. 이 협회는 산림이 자원으로서의 가치뿐만 아니라, 수자원 보호, 기후 조절, 공기 정화 등 국민의 건강과 복지에 미치는 영향을 고려하여 국민의 복리를 도모하고자 하였습니다. 특히, 당시 한국은 무분별한 벌목으로 인해 산림이 황폐해지고 있었고, 이에 대한 경각심을 일깨우기 위해 협회가 설립된 것입니다. 협회는 산림 사업이 실업계에서 긴급한 과제임을 강조하며, 국민의 참여와 지지를 요청하였습니다.'}

# 4. Gemma 파인튜닝

#### 주의: Colab GPU 메모리 한계로 이전장 추론에서 사용했던 메모리를 비워 줘야 파인튜닝을 진행 할 수 있습니다. <br> notebook 런타임 세션을 재시작 한 후 1번과 2번의 2.1 항목까지 다시 실행하여 로드 한 후 아래 과정을 진행합니다

In [9]:
!nvidia-smi

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Wed Sep 18 12:24:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 4.1 학습용 프롬프트 조정

In [10]:
def generate_prompt(example):
    prompt_list = []
    for question, answer in zip(example['Question'], example['Answer']):
        prompt = (f"<bos><start_of_turn>user\n"
                  f"{question}<end_of_turn>\n"
                  f"<start_of_turn>model\n{answer}<end_of_turn><eos>")
        prompt_list.append(prompt)
    return prompt_list

In [11]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
대한국산림협회가 설립된 배경과 그 중요성은 무엇인가요?<end_of_turn>
<start_of_turn>model
대한국산림협회는 산림의 중요성을 인식하고,植林(식림)의 필요성을 강조하기 위해 설립되었습니다. 이 협회는 산림이 자원으로서의 가치뿐만 아니라, 수자원 보호, 기후 조절, 공기 정화 등 국민의 건강과 복지에 미치는 영향을 고려하여 국민의 복리를 도모하고자 하였습니다. 특히, 당시 한국은 무분별한 벌목으로 인해 산림이 황폐해지고 있었고, 이에 대한 경각심을 일깨우기 위해 협회가 설립된 것입니다. 협회는 산림 사업이 실업계에서 긴급한 과제임을 강조하며, 국민의 참여와 지지를 요청하였습니다.<end_of_turn><eos>


### 4.2 QLoRA 설정

In [12]:
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

In [13]:
BASE_MODEL = "google/gemma-2-2b-it"
# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config, attn_implementation="eager",)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", attn_implementation="eager",)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 4.3 Trainer 실행

In [19]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=50,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        # warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        push_to_hub=True,
        report_to='wandb',
        run_name="gemma_sprint",  # name of the W&B run (optional)
        logging_steps=10,
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/acce

In [20]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=10, training_loss=2.18530387878418, metrics={'train_runtime': 15.437, 'train_samples_per_second': 2.591, 'train_steps_per_second': 0.648, 'total_flos': 112232033851392.0, 'train_loss': 2.18530387878418, 'epoch': 0.004471272076905879})

### 4.4 Finetuned Model 저장

In [ ]:
# ADAPTER_MODEL = "lora_adapter"

# trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
# !ls -alh lora_adapter

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
# model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

# model = model.merge_and_unload()
# model.save_pretrained('gemma-2b-it-sum-ko')

In [ ]:
# !ls -alh ./gemma-2b-it-sum-ko

# 5. Gemma 한국어 요약 모델 추론

#### 주의: 마찬가지로 Colab GPU 메모리 한계로 학습 시 사용했던 메모리를 비워 줘야 파인튜닝을 진행 할 수 있습니다. <br> notebook 런타임 세션을 재시작 한 후 1번과 2번의 2.1 항목까지 다시 실행하여 로드 한 후 아래 과정을 진행합니다

In [ ]:
# !nvidia-smi

### 5.1 Fine-tuned 모델 로드

In [ ]:
# BASE_MODEL = "google/gemma-2b-it"
# FINETUNE_MODEL = "./gemma-2b-it-sum-ko"

# finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

### 5.2 Fine-tuned 모델 추론

In [ ]:
# pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
# question = dataset['train']['Question'][1000]
# answer = dataset['train']['Answer'][1000]

In [ ]:
# messages = [
#     # {
#     #     "role": "system",
#     #     "content": "You are a helpful assistant."
#     # },
#     {
#         "role": "user",
#         "content": f"{question}"
#     },

# ]
# prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
# outputs = pipe_finetuned(
#     prompt,
#     do_sample=True,
#     temperature=0.1,
#     top_k=50,
#     top_p=0.95,
#     add_special_tokens=True
# )
# print(outputs[0]["generated_text"][len(prompt):])